In [10]:
type_num=14
date='01.02'
trainingSet_path0= "./data/train_"+date+".txt"
valSet_path0= "./data/test_"+date+".txt"
bad_path0= "./data/bad_title_"+date+".txt"

type_=['ICT', '新能源汽车', '生物医药', '医疗器械', '钢铁', 
       '能源', '工业机器人', '先进轨道交通', '数控机床',  '工业软件', 
       '高端装备', '半导体', '人工智能', '稀土']
www=[3.6275337837837838, 1.2170586568432984, 4.9768250289687135, 11.79945054945055, 8.555776892430279, 2.6826983135540288, 1.0, 10.818639798488665, 7.134551495016611, 21.262376237623762, 13.09451219512195, 30.246478873239436, 20.070093457943926, 1.9792626728110598]


In [11]:
def read_list(text_path):
    lsit=[]
    with open('%s' % text_path, 'r', encoding="utf8") as f:  # 打开一个文件只读模式
        line = f.readlines()  # 读取文件中的每一行，放入line列表中
        for line_list in line:
            lsit.append(line_list.replace('\n',''))
    return lsit

bad_title=read_list(bad_path0)
bad_title[0]

'11月12日午间公告一览:信隆健康涉及的美国消费者诉讼案达成和解'

In [13]:
import pandas as pd
def make_data(path):
    data=pd.read_table(path,header=None)#text label
    data.columns = ['text', 'label']
    text=[i for i in data['text']]
    label=[i for i in data['label']]
    return text,label

train_text,train_label=make_data(trainingSet_path0)
test_text,test_label=make_data(valSet_path0)

train_text[0],train_label[0],test_text[0],test_label[0]

('军工行业：军工批量采购唱响需求号角', 6, '语音识别大拿DanielPovey发布新一代框架Kaldi，有望年内应用于小米产品', 0)

In [15]:
import re
import jieba
import pickle
from sklearn.feature_extraction.text import CountVectorizer
import joblib
import numpy as np
#from utils import tokenize, load_curpus
def tokenize(text):
    """
    带有语料清洗功能的分词函数
    """
    text = re.sub("\{%.+?%\}", " ", text)           # 去除 {%xxx%} (地理定位, 微博话题等)
    text = re.sub("@.+?( |$)", " ", text)           # 去除 @xxx (用户名)
    text = re.sub("【.+?】", " ", text)              # 去除 【xx】 (里面的内容通常都不是用户自己写的)
    icons = re.findall("\[.+?\]", text)             # 提取出所有表情图标
    text = re.sub("\[.+?\]", "IconMark", text)      # 将文本中的图标替换为`IconMark`

    tokens = []
    for k, w in enumerate(jieba.lcut(text)):
        w = w.strip()
        if "IconMark" in w:                         # 将IconMark替换为原图标
            for i in range(w.count("IconMark")):
                tokens.append(icons.pop(0))
        elif w and w != '\u200b' and w.isalpha():   # 只保留有效文本
                tokens.append(w)
    return tokens


def load_curpus(path):
    """
    加载语料库
    """
    data = []
    with open(path, "r", encoding="utf8") as f:
        for line in f:
            [_, seniment, content] = line.split(",", 2)
            content = tokenize(content)             # 分词
            data.append((content, int(seniment)))
    return data



In [19]:
train_tokenize=[tokenize(i) for i in train_text]
test_tokenize=[tokenize(i) for i in test_text]
print(train_tokenize[0],test_tokenize[0])

['军工', '行业', '军工', '批量', '采购', '唱响', '需求', '号角'] ['语音', '识别', '大', '拿', 'DanielPovey', '发布', '新一代', '框架', 'Kaldi', '有望', '年内', '应用', '于', '小米', '产品']


In [23]:
train_content=[" ".join(content) for content in train_tokenize]
test_content=[" ".join(content) for content in test_tokenize]
print(train_content[0],'///',test_content[0])

军工 行业 军工 批量 采购 唱响 需求 号角 /// 语音 识别 大 拿 DanielPovey 发布 新一代 框架 Kaldi 有望 年内 应用 于 小米 产品


In [24]:
stopwords = []
with open("./data/stopwords.txt", "r", encoding="utf8") as f:
    for w in f:
        stopwords.append(w.strip())

In [25]:
from sklearn.feature_extraction.text import CountVectorizer
data_str = train_content+test_content
vectorizer = CountVectorizer(token_pattern='\[?\w+\]?', stop_words=stopwords)
vectorizer.fit_transform(data_str)

<16393x20492 sparse matrix of type '<class 'numpy.int64'>'
	with 146957 stored elements in Compressed Sparse Row format>

In [26]:
X_train = vectorizer.transform(train_content)
y_train = train_label
print(X_train.shape)
print(len(y_train))
print(y_train[0])
X_test = vectorizer.transform(test_content)
y_test=test_label
print(X_test.shape)
print(len(y_test))
print(y_test[0])

(11475, 20492)
11475
6
(4918, 20492)
4918
0


In [66]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train, y_train)
y_pre = clf.predict(X_test)
y_pre_proba= clf.predict_proba(X_test)

In [67]:
import copy
y_pre0=copy.deepcopy(y_pre)
from sklearn import metrics
for i in range(len(y_pre)):
    if y_pre_proba[i][y_pre[i]]<0.75:
        y_pre0[i]=-1
print(metrics.classification_report(y_test, y_pre0))
print("准确率:", metrics.accuracy_score(y_test, y_pre0))

              precision    recall  f1-score   support

          -1       0.00      0.00      0.00         0
           0       0.82      0.41      0.54       345
           1       0.89      0.86      0.88      1013
           2       0.97      0.85      0.90       251
           3       0.94      0.53      0.68       115
           4       1.00      0.67      0.80       151
           5       0.96      0.79      0.87       506
           6       0.93      0.97      0.95      1277
           7       0.99      0.68      0.81       130
           8       0.93      0.54      0.69       181
           9       1.00      0.38      0.55        50
          10       1.00      0.71      0.83       114
          11       1.00      0.24      0.38        42
          12       1.00      0.48      0.65        67
          13       0.95      0.97      0.96       676

    accuracy                           0.82      4918
   macro avg       0.89      0.61      0.70      4918
weighted avg       0.93   

/data/fuwen/anaconda3/envs/sw0/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/data/fuwen/anaconda3/envs/sw0/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/data/fuwen/anaconda3/envs/sw0/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [48]:
for i in range(len(y_pre)):
    print(test_text[i]+'\t标注：'+type_[y_test[i]]+'\t预测：'+type_[y_pre[i]])
    print(y_pre_proba[i][y_pre[i]])

语音识别大拿DanielPovey发布新一代框架Kaldi，有望年内应用于小米产品	标注：ICT	预测：新能源汽车
0.7098478707597217
吹响稀土业复工复产集结号	标注：稀土	预测：稀土
0.9999840187497028
能检测机器人助力成都动车组，列车“体检”时间节约一半	标注：工业机器人	预测：工业机器人
0.9985699998920259
突发！微软亚研CV大牛王井东离职，或将加入百度	标注：ICT	预测：ICT
0.995039019243845
国际能源网-风电每日报，3分钟·纵览风电事！（9月17日）	标注：能源	预测：能源
1.0
CheckPoint计划在亚太地区引进100名青年人才	标注：工业机器人	预测：ICT
0.30938141144314696
宝马总部四缸大厦照亮慕尼黑夜空	标注：新能源汽车	预测：新能源汽车
0.9806759777672526
中核汇能发布2021年5GW组件二次集采招标	标注：能源	预测：能源
0.9987264906743383
微创医疗机器人在港上市，开盘竟遭破发！	标注：工业机器人	预测：工业机器人
0.9998717119498896
擎朗智能完成近亿元C轮融资用于发展配送机器人	标注：工业机器人	预测：工业机器人
0.9999924725685593
首款！地舒单抗生物类似药报上市来自齐鲁制药	标注：生物医药	预测：生物医药
0.9999920364410619
周迅成为五菱品牌全球代言人：“认真做事”价值观高度一致	标注：新能源汽车	预测：新能源汽车
0.9845617676367833
小冰框架下人工智能小艺术家COCO获国际艺术节大奖	标注：半导体	预测：半导体
0.7560697103859016
工业机器人绝对精度问题探讨	标注：工业机器人	预测：工业机器人
0.9998494476059655
复盘支撑特斯拉疯狂降价16万背后的6家机器人供应商	标注：工业机器人	预测：工业机器人
0.9725688268635632
芯片危机风暴中的汽车市场奥迪传停产风波	标注：新能源汽车	预测：新能源汽车
0.9991127736596506
博世高管：汽车行业半导体供应链已经崩溃	标注：新能源汽车	预测：新能源汽车
0.928541000482524
参会指南：2021年全国农机流通工

0.9848840659506435
新高！华润电力：新增风光并网装机量4.48GW	标注：能源	预测：能源
0.999999712001922
『波黑』萨拉热窝有轨电车轨道重建项目标段正式动工	标注：先进轨道交通	预测：先进轨道交通
0.7511305925519227
曾经的国货经典瑞风汽车持续发力成为国潮新标杆	标注：新能源汽车	预测：新能源汽车
0.9938243681237846
Flexiv自适应机器人拂晓成为业内首个获得CE+ETL双认证的力控型机器人	标注：工业机器人	预测：工业机器人
0.9999942463496121
『贵南高铁』全线首座特大型隧道顺利贯通	标注：先进轨道交通	预测：先进轨道交通
0.9999992600020315
把握智能网联行业机遇业内专家有话说	标注：工业软件	预测：新能源汽车
0.960235064039828
全球扫地机器人2019年市场规模达33亿美元，中国占一半	标注：工业机器人	预测：工业机器人
0.9999972025837044
英国NHS引入AI技术辅助动脉瘤手术规划，准确率达96.75%	标注：ICT	预测：工业机器人
0.9954792669809314
对赌协议未履约！科华恒盛将石军、肖贵阳等告上法庭	标注：能源	预测：先进轨道交通
0.35879310673610626
日本电动化教训：9年前充电桩布局盲目追求数量导致浪费	标注：新能源汽车	预测：新能源汽车
0.9999841183915023
27日中国稀土价格指数微幅上涨	标注：稀土	预测：稀土
0.999989642535617
香港城市大学开发了新一代微针技术：高效递送DC疫苗，显著延缓肿瘤生长	标注：生物医药	预测：生物医药
0.978078632450643
国轩高科：拟不超2.6亿元投建国轩1GWh高性能电芯项目	标注：ICT	预测：新能源汽车
0.8627769534523575
大族机器人：协作机器人是机器人发展的终极形态吗？	标注：工业机器人	预测：工业机器人
0.9999999997591118
喜茶店员拿错饮料致顾客喝下后洗胃官方致歉	标注：新能源汽车	预测：ICT
0.35418827419838694
E周看点丨广汽埃安拟开展混改、8月造车新势力排位再生变	标注：新能源汽车	预测：新能源汽车
0.99808043035

1.0
2021第十一届中国国际机器人高峰论坛暨第七届恰佩克颁奖仪式在芜湖开幕	标注：工业机器人	预测：工业机器人
0.9999815457943477
八大诱因！国网电科院解析北京储能电站爆炸事故	标注：能源	预测：能源
0.995476374509415
2021年8月|NMPA批准的新药	标注：生物医药	预测：生物医药
0.9382192852455935
中国航发燃机——借鉴燃气轮机“欧盟氢能战略”	标注：工业软件	预测：工业软件
0.7619556559566789
电力股继续杀跌，原因是啥？	标注：能源	预测：能源
0.9345555016715367
平价时代！江苏中车推进产品全生命周期降本	标注：能源	预测：能源
0.9797183146975984
完成数千万元融资，以领先性能落地复杂场景应用，构筑强护城河	标注：工业机器人	预测：工业机器人
0.9939540577549406
浙江对维护医保信息编码标准药品代码信息的再次提醒通知	标注：生物医药	预测：生物医药
0.9992585395449827
软体机器人成“挖洞专家”	标注：工业机器人	预测：工业机器人
0.9998504144605842
国产码垛机器人的特点及应用领域	标注：工业机器人	预测：工业机器人
0.9999780323971182
生态环境部：推动PM2.5与臭氧浓度下降推动钢铁行业超低排放改造	标注：钢铁	预测：钢铁
0.998620319669183
2021年中国工业机器人远程前程，应用市场遍地开花	标注：工业机器人	预测：工业机器人
0.9999812114312278
2021年全球工业机器人行业技术竞争格局	标注：工业机器人	预测：工业机器人
0.9999990401162566
防城港市中医医院肝功能剪切波量化超声诊断仪谈判公告	标注：医疗器械	预测：医疗器械
0.999933024591439
陕西第五批国家组织药品集中采购中选供应品种补充信息	标注：生物医药	预测：生物医药
0.9999998166297556
国产外骨骼机器人问世，登山、徒步、搬运、养老……大有用武之地	标注：工业机器人	预测：工业机器人
0.9999596894614771
科技助力防疫：高性能防疫消毒机器人今天“上岗”	标注：工业机器人	预测：工业机器人
0.9999999903081

面向千亿机器人产业总部碧桂园总部新翼智谷获百余企业家追捧	标注：工业机器人	预测：工业机器人
0.9999999975135836
南通振康焊接机电有限公司参评“维科杯·OFweek2020中国机器人行业卓越供应商奖”	标注：工业机器人	预测：工业机器人
1.0
市场急速回升人福涨了八成！口服制剂“内卷”	标注：生物医药	预测：稀土
0.36267510071899023
上海快仓智能科技有限公司推进智慧物流创新实践——机器人“跑腿”显身手	标注：工业机器人	预测：工业机器人
0.9999999294729089
ABB将收购ASTI移动机器人，以自主移动机器人推动柔性自动化迈入新阶段	标注：工业机器人	预测：工业机器人
0.9999999926149172
首座极充站将于9月落成极氪子品牌极氪能源亮相	标注：新能源汽车	预测：新能源汽车
0.9978614058330773
鞍钢股份、马鞍山钢铁股份高层有调整	标注：钢铁	预测：钢铁
0.9890101024208845
Oxbotica与AppliedEV合作开发多用途无人驾驶汽车	标注：新能源汽车	预测：新能源汽车
0.6232112985921071
工信部副部长辛国斌：中国新能源车成本偏高，将统筹保障动力电池资源供应	标注：新能源汽车	预测：新能源汽车
0.9999646347398992
机器人，为“刚需”而来	标注：工业机器人	预测：工业机器人
0.9969603476647526
营收、利润双增长！亨通光电2020年报&2021一季度财报出炉	标注：能源	预测：能源
0.9214825611352572
陈希颖:稀土之光点亮人生	标注：稀土	预测：稀土
0.9900288746864693
上海宾通智能科技有限公司参评“维科杯·OFweek2020中国机器人行业优秀产品奖”	标注：工业机器人	预测：工业机器人
1.0
通用现代电动车起火电池系LG亚洲两工厂生产	标注：新能源汽车	预测：新能源汽车
0.9999994099084155
中国稀土行业协会到访中国稀有稀土	标注：稀土	预测：稀土
0.9999999980640837
MOX公司MOSAICSCADA软件在南水北调工程中应用	标注：高端装备	预测：高端装备
0.9693742833193024
南京埃斯顿自动化股份有限公司参评“维科杯·OFw

河北省构建降碳产品价值实现机制	标注：能源	预测：稀土
0.43330046118198917
麻省理工开发自动驾驶机器人船部署在阿姆斯特丹运河	标注：工业机器人	预测：工业机器人
0.7855719512362346
新松协作机器人集体亮相，全新HC-X1型复合机器人重磅来袭	标注：工业机器人	预测：工业机器人
0.9999999892105934
苏州灵猴机器人有限公司参评“维科杯·OFweek2020中国机器人行业优秀产品奖”	标注：工业机器人	预测：工业机器人
1.0
今日能源看点：国家能源局：2020年风光新增装机近120GW	标注：能源	预测：能源
0.9999999997779412
OFweek2021机器人产业大会开幕在即德鲁动力将与业界共襄盛举	标注：工业机器人	预测：工业机器人
0.9999766575351321
政协委员、正泰集团南存辉：围绕电力新能源主方向做文章，推动绿色转型新发展	标注：能源	预测：能源
0.9676760267161849
今日能源看点：《山东省煤电应急备用电源方案》出炉	标注：能源	预测：能源
0.9999999671781273
自媒体说车恶意中伤国产车企！吉利杨学良：已被封禁并道歉	标注：新能源汽车	预测：新能源汽车
0.8073692404362888
宝马将在两年内发布4款纯电动汽车	标注：新能源汽车	预测：新能源汽车
0.9996563796769078
2021年，巡检机器人三大应用还将迎来爆发！	标注：工业机器人	预测：工业机器人
0.9999844475006155
法雷奥携六项创新技术亮相2021IAAMobility德国国际汽车及智慧出行展，助力更安全更清洁更多元的移动出行	标注：新能源汽车	预测：新能源汽车
0.998156807176243
国际首套！百兆瓦先进压缩空气储能示范项目获重大进展	标注：能源	预测：能源
0.9966068446267871
稀土市场开始触底反弹	标注：稀土	预测：稀土
0.9999754835593658
盖世研究院韩亦可：至2025中国汽车座椅市场规模保持在800-1000亿	标注：新能源汽车	预测：工业机器人
0.4729986298460625
俄罗斯网红粘汽车外！时速180公里高速飞驰	标注：新能源汽车	预测：新能源汽车
0.998054264753866

In [32]:
bad_vec=vectorizer.transform(bad_title)
bad_pre = clf.predict(bad_vec)
bad_pre_proba = clf.predict_proba(bad_vec)


In [38]:
for i in range(len(bad_title)):
    print(bad_title[i],type_[bad_pre[i]],str(bad_pre_proba[i][bad_pre[i]]))
    print([type_[j]+':'+str(bad_pre_proba[i][j]) for j in range(len(type_))])
    print('___________')

11月12日午间公告一览:信隆健康涉及的美国消费者诉讼案达成和解 工业机器人 0.2630065359477121
['ICT:0.07311546840958608', '新能源汽车:0.21925925925925946', '生物医药:0.05333333333333336', '医疗器械:0.021699346405228782', '钢铁:0.030588235294117676', '能源:0.09542483660130718', '工业机器人:0.2630065359477121', '先进轨道交通:0.023267973856209163', '数控机床:0.0366884531590414', '工业软件:0.013246187363834431', '高端装备:0.018649237472766887', '半导体:0.008714596949891075', '人工智能:0.012810457516339873', '稀土:0.13019607843137262']
___________
路透社：香港大学新课程列出违反国安法的危险 新能源汽车 0.2903477054405629
['ICT:0.07463231700933368', '新能源汽车:0.2903477054405629', '生物医药:0.05874852908148449', '医疗器械:0.027998543883191375', '钢铁:0.037162716555500214', '能源:0.09050402509333105', '工业机器人:0.1591451704508698', '先进轨道交通:0.029669683647745894', '数控机床:0.043403269500138085', '工业软件:0.0176373100856867', '高端装备:0.024532638752267733', '半导体:0.011916645904122591', '人工智能:0.017143168008234323', '稀土:0.11715827658753208']
___________
百济Q3财报：PD1单抗中国销售7700万美元研发支出3.519亿美元 工业机器人 0.2630065359477121
['ICT:0.073115468409586

___________
启动L4级无人卡车研发，阿里加速布局无人物流赛道 工业机器人 0.2630065359477121
['ICT:0.07311546840958608', '新能源汽车:0.21925925925925946', '生物医药:0.05333333333333336', '医疗器械:0.021699346405228782', '钢铁:0.030588235294117676', '能源:0.09542483660130718', '工业机器人:0.2630065359477121', '先进轨道交通:0.023267973856209163', '数控机床:0.0366884531590414', '工业软件:0.013246187363834431', '高端装备:0.018649237472766887', '半导体:0.008714596949891075', '人工智能:0.012810457516339873', '稀土:0.13019607843137262']
___________
领克03+Cyan定制版上市：2.0T动力更强23.68万起售 工业机器人 0.2630065359477121
['ICT:0.07311546840958608', '新能源汽车:0.21925925925925946', '生物医药:0.05333333333333336', '医疗器械:0.021699346405228782', '钢铁:0.030588235294117676', '能源:0.09542483660130718', '工业机器人:0.2630065359477121', '先进轨道交通:0.023267973856209163', '数控机床:0.0366884531590414', '工业软件:0.013246187363834431', '高端装备:0.018649237472766887', '半导体:0.008714596949891075', '人工智能:0.012810457516339873', '稀土:0.13019607843137262']
___________
人事变动|原长安总裁周治平任一汽集团党委常委、副总经理 新能源汽车 0.5356018690226106
['ICT:0.06367354

___________
“美国队长”和“黑寡妇”将出演苹果AppleTV+浪漫动作电影《幽灵》 ICT 0.29916757179810133
['ICT:0.29916757179810133', '新能源汽车:0.09436888088309389', '生物医药:0.06353385523110114', '医疗器械:0.03546779426140553', '钢铁:0.04432721292265397', '能源:0.0842721950698453', '工业机器人:0.09454320666534086', '先进轨道交通:0.03714301950913301', '数控机床:0.05041101380323802', '工业软件:0.023055988629531756', '高端装备:0.03168380930194902', '半导体:0.01599818468012802', '人工智能:0.022523063664392058', '稀土:0.1035042035800867']
___________
台积电明年营收有望增22-23%，毛利率超英特尔 工业机器人 0.2630065359477121
['ICT:0.07311546840958608', '新能源汽车:0.21925925925925946', '生物医药:0.05333333333333336', '医疗器械:0.021699346405228782', '钢铁:0.030588235294117676', '能源:0.09542483660130718', '工业机器人:0.2630065359477121', '先进轨道交通:0.023267973856209163', '数控机床:0.0366884531590414', '工业软件:0.013246187363834431', '高端装备:0.018649237472766887', '半导体:0.008714596949891075', '人工智能:0.012810457516339873', '稀土:0.13019607843137262']
___________
美团高管解读Q2财报：将积极履行职责，更加努力保护骑手权利 工业机器人 0.2630065359477121
['ICT:0.07311546

流亡藏人团体：有上千名西藏政治犯仍在狱中 工业机器人 0.2630065359477121
['ICT:0.07311546840958608', '新能源汽车:0.21925925925925946', '生物医药:0.05333333333333336', '医疗器械:0.021699346405228782', '钢铁:0.030588235294117676', '能源:0.09542483660130718', '工业机器人:0.2630065359477121', '先进轨道交通:0.023267973856209163', '数控机床:0.0366884531590414', '工业软件:0.013246187363834431', '高端装备:0.018649237472766887', '半导体:0.008714596949891075', '人工智能:0.012810457516339873', '稀土:0.13019607843137262']
___________
维吾尔团体拟就中国侵犯人权行为向阿根廷法院申诉 工业机器人 0.2630065359477121
['ICT:0.07311546840958608', '新能源汽车:0.21925925925925946', '生物医药:0.05333333333333336', '医疗器械:0.021699346405228782', '钢铁:0.030588235294117676', '能源:0.09542483660130718', '工业机器人:0.2630065359477121', '先进轨道交通:0.023267973856209163', '数控机床:0.0366884531590414', '工业软件:0.013246187363834431', '高端装备:0.018649237472766887', '半导体:0.008714596949891075', '人工智能:0.012810457516339873', '稀土:0.13019607843137262']
___________
Omicron构成“非常高”的全球风险关于其严重性的数据有限--世卫组织 工业机器人 0.2630065359477121
['ICT:0.07311546840958608', '新能

In [68]:
import joblib
joblib.dump(clf,'./data/bayes_'+date+'.pkl')

import pickle
feature_path='./data/feature_'+date+'.pkl'
with open(feature_path,'wb') as fw:
    pickle.dump(vectorizer.vocabulary_,fw)